Pytorch에서 제공하는 RNN 클래스는 기본적으로 Elman RNN 이지만,
제공하는 클래스 대신 직접 Elman RNN을 구현한다.
하나의 RNN 타임 스텝을 구현한 RNNCell을 사용하여 Elman RNN을 구현해보자.

In [ ]:
import numpy as np
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
class ElmanRNN(nn.Module):
  def __init__(self, input_size, hidden_size, batch_first=False): # input_size : 입력 벡터 크기, hidden_size : 은닉 상태 벡터 크기, batch_first : 0번째 차원이 배치인지 여부
    super(ElmanRNN, self).__init__()
    self.rnn_cell = nn.RNNCell(input_size, hidden_size)

    self.batch_first = batch_first  # 0번째 차원이 배치인지 여부
    self.hidden_size = hidden_size  # 은닉 상태 벡터 크기

  def _initialize_hidden(self, batch_size):
    return torch.zeros((batch_size, self.hidden_size)) # 형상은 (N x H)가 되며, N : 배치 크기, H :은닉 상태 크기

  def forward(self, x_in, initial_hidden=None):
    '''
    0번째 차원이 배치라면 x_in의 0번째와 1번째 순서를 바꾸어준다.
    반대로 0번째 차원이 배치가 아닌 경우에는
    0번째 차원 : 시퀀스 크기 (시계열 데이터의 분량)
    1번째 차원 : 배치 크기
    '''
    if self.batch_first:
      batch_size, seq_size, feat_size = x_in.size()
      x_in = x_in.permute(1, 0, 2)
    else:
      seq_size, batch_size, feat_size = x_in.size()

    hiddens = []

    if initial_hidden is None:                    # 초기 은닉이 정해져있지 않다면, 초기화한다.
      initial_hidden = self._initialize_hidden(batch_size)
      initial_hidden = initial_hidden.to(x_in.device)

    hidden_t = initial_hidden

    for t in range(seq_size):
      hidden_t = self.rnn_cell(x_in[t], hidden_t) # 입력 벡터와 은닉 상태를 전달하는데, 출력된 은닉 상태는 다음의 rnn_cell에 전달된다.
      hiddens.append(hidden_t)

    hiddens = torch.stack(hiddens)

    if self.batch_first:                          # 0번째 차원이 배치라면 은닉 상태 순서를 바꿔준다.
      hiddens = hiddens.permute(1, 0, 2)

    return hiddens